<a href="https://colab.research.google.com/github/himankgalundia/AI-chess-bot-/blob/main/ChessPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip3 install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=c63e029fb112fd1ebfc7d0d059a96b9e9b5cce93f566837338b766f3fae83b5e
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import patoolib
import chess
from sklearn.model_selection import train_test_split

In [ ]:
example_fen = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [ ]:
def bitMap(fen):
  pieces = [chess.PAWN, chess.BISHOP, chess.KNIGHT, chess.ROOK, chess.QUEEN, chess.KING]
  result = []
  for turn in [chess.WHITE, chess.BLACK]:
    for piece in pieces:
      board = chess.Board(fen)
      piecesArray = board.pieces(piece, turn)
      arr = np.array(piecesArray.tolist())
      arr = np.reshape(arr, (8, 8)).astype(int)
      result.append(arr)
  result = np.transpose(result, axes=(1, 2, 0)) #magic
  return result

In [ ]:
data = pd.read_csv('chessData.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'chessData.csv'

In [ ]:
data.head()

TIME TO GET TRAINING DATA

In [ ]:
amountToTrainOn = 100
X = np.zeros((amountToTrainOn, 8, 8, 12))
Y = np.zeros((amountToTrainOn,))
for i in range(amountToTrainOn):
  fen = data.iloc[i]['FEN']
  eval = data.iloc[i]['Evaluation']
  if '#' in eval:
    eval = eval[1:]
  bitmap = bitMap(fen)
  X[i] = bitmap
  Y[i] = int(eval)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
input = tf.keras.layers.Input((8, 8, 12))
x1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputt)
x2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x1)
x3 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x2)
x4 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x3)
x5 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x4)
f = tf.keras.layers.Flatten()(x5)
d1 = tf.keras.layers.Dense(64, activation='relu')(f)
d2 = tf.keras.layers.Dense(32, activation='relu')(d1)
d3 = tf.keras.layers.Dense(1)(d2)
model = tf.keras.models.Model(inputs=inputt, outputs=d3)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])
model.fit(x_train,y_train, epochs = 100, batch_size=128, validation_data = (x_test, y_test))

In [ ]:
predictions = model.predict(x_test)
for i in range(len(predictions)):
  print('Prediction: ' + str(predictions[i]) + ', Actual: ' + str(y_test[i]))

In [ ]:
def findMove(fen, turn='white'):
  board = chess.Board(fen)
  turn = 'white'
  if turn == 'white':
    bestMove = ''
    bestEval = -1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitMap(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval > bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  else:
    bestMove = ''
    bestEval = 1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitMap(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval < bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  return bestMove

In [ ]:
startBoard = chess.Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
print(startBoard)
userinput = input('Enter your first move: ')
while userinput != 'end':
  move = chess.Move.from_uci(userinput)
  startBoard.push(move)
  botMove = findMove(startBoard.fen(), turn = 'black')
  startBoard.push(botMove)
  print()
  print()
  print(startBoard)
  userinput = input('Enter your next move: ')

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . P . . . . .
. P P P P P P P
R N B Q K B N R


KeyboardInterrupt: Interrupted by user